In [ ]:
'''

以先前案例組合而成

S3 客戶端生成
DynamoDB 客戶端生成
flask app 生成

創建S3 桶子
創建 DynamoDB Table

第一個方法
    用戶訪問網站
    看見上傳網站頁面，下方有目前已上傳圖片的表格清單，表格內要有 S3 的連結名

第二個方法
    用戶上傳圖片
    伺服器收到 Request
    生成 presigned url，上傳的檔案預設需公開
    並插入資料至 DynamoDB
    讓用戶將圖片傳至 presigned url
    看見上傳網站頁面，下方有目前已上傳圖片的表格清單，表格內要有 S3 的連結名


啟動flask app，使外部port 5000 能訪問


'''

In [ ]:
!pip install boto3 awscli requests flask

In [ ]:
'''

S3 客戶端生成
DynamoDB 客戶端生成
flask app 生成

'''
import boto3
from flask import Flask

s3_client = boto3.client('s3', endpoint_url='http://localstack-main:4566')

dynamodb = boto3.resource('dynamodb', endpoint_url='http://localstack-main:4566')

app = Flask(__name__)


In [ ]:
'''

創建S3 桶子
創建 DynamoDB Table

'''


create_bucket_response = s3_client.create_bucket(
    Bucket='cxcxc-aws-certificate',
)
print(create_bucket_response)


table = dynamodb.create_table(
    TableName='s3-objects-list', 
    KeySchema=[
        {
            'AttributeName': 'image_name',
            'KeyType': 'HASH'  # Partition key
        },
        {
            'AttributeName': 'upload_time',
            'KeyType': 'RANGE'  # Sort key
        }
    ], 
    AttributeDefinitions=[
        {
            'AttributeName': 'image_name',
            'AttributeType': 'S'  # N 代表數字； S 代表字串。
        },
        {
            'AttributeName': 'upload_time',
            'AttributeType': 'S'  
        },
    ],
    ProvisionedThroughput={
        'ReadCapacityUnits': 5,
        'WriteCapacityUnits': 5
    }
)

print(table)


In [ ]:
# 列出現有 bucket
response = s3_client.list_buckets()
print(response.get('Buckets'))

# 列出現有 table
table_list = list(dynamodb.tables.all())
print(table_list)


In [ ]:
'''

第一個方法
    用戶訪問網站
    看見上傳網站頁面，下方有目前已上傳圖片的表格清單，表格內要有 S3 的連結名

'''
import os
from flask import render_template, request, redirect, url_for
from werkzeug.utils import secure_filename
from boto3.dynamodb.conditions import Key, Attr
import datetime
from pprint import pprint


@app.route('/images', methods=['GET'])
def list_images():
    # 生成指定 table 物件
    table = dynamodb.Table('s3-objects-list')

    # 掃描 DynamoDB 將所有 image 列出
    response = table.scan(
        FilterExpression=Attr('image_name').exists()
    )
    items = response['Items']
    
    # 準備渲染在頁面上的資料list
    uploaded_images_list = []
    images_url_list = []
    for i in items:
        uploaded_images_list.append(i.get('image_name'))
        images_url_list.append(i.get('image_url'))
        
    return render_template(
        'images.html',
        length = list(range(table.item_count)),
        uploaded_images = uploaded_images_list,
        images_url = images_url_list
    )
    

In [ ]:
'''

第二個方法
    用戶上傳圖片
    伺服器收到 Request
    生成 presigned url，上傳的檔案預設需公開
    並插入資料至 DynamoDB
    讓用戶將圖片傳至 presigned url
    看見上傳網站頁面，下方有目前已上傳圖片的表格清單，表格內要有 S3 的連結名

'''
import os

cwd = os.getcwd() 

UPLOAD_FOLDER = os.path.join(cwd, 'uploaded_images')

print(UPLOAD_FOLDER)

@app.route('/upload-images', methods=['GET', 'POST'])
def upload_images():
    # 生成指定 table 物件
    table = dynamodb.Table('s3-objects-list')
    
    if request.method == 'POST':
        # 從 request 取得檔案
        file = request.files.get('file')
        
        # 取得檔案名稱與合成存檔路徑
        filename = secure_filename(file.filename)
        file_path = os.path.join(UPLOAD_FOLDER, filename)
        
        # 將照片存檔
        file.save(file_path)
        
        # 上傳照片至S3
        s3_client.upload_file(file_path, 
                             'cxcxc-aws-certificate', 
                             filename)
        
        # 取得剛上傳照片的 presigned url
        presign_url = s3_client.generate_presigned_url(
            'get_object',
            Params={
                'Bucket': 'cxcxc-aws-certificate',
                'Key': filename
            }
        )
        
        # 將照片名稱、上傳時間、presigned url 記錄在 dynamodb
        table.put_item(
            Item={"image_name": filename,
                  "upload_time": str(datetime.datetime.utcnow()),
                  "image_url": presign_url}
        )
    
    
    # 以下與第一個方法相同
    
    # 掃描DynamoDB 將所有image 列出
    response = table.scan(
        FilterExpression=Attr('image_name').exists()
    )
    items = response['Items']
    
    # 準備渲染在頁面上的資料list
    uploaded_images_list = []
    images_url_list = []
    for i in items:
        uploaded_images_list.append(i.get('image_name'))
        images_url_list.append(i.get('image_url'))
        
    return render_template(
        'upload-images.html',
        length = list(range(table.item_count)),
        uploaded_images = uploaded_images_list,
        images_url = images_url_list
    )
    

In [ ]:
'''

啟動flask app，使外部port 5000 能訪問

'''

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)